<a href="https://colab.research.google.com/github/FilipKatulski/machine-learning/blob/lab2-dev/UM_LAB_02/UM_LAB_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt install swig cmake libopenmpi-dev zlib1g-dev
!pip install stable-baselines[mpi]==2.10.0 box2d box2d-kengz
!pip install Box2D
!pip install box2d-py
!pip install gym[all]
!pip install gym[Box_2D]
!pip3 install imageio==2.4.1
!apt-get install -y xvfb python-opengl ffmpeg
!pip install ffmpeg-python
!pip install -U colabgymrender


Reading package lists... Done
Building dependency tree       
Reading state information... Done
libopenmpi-dev is already the newest version (2.1.1-8).
cmake is already the newest version (3.10.2-1ubuntu2.18.04.2).
zlib1g-dev is already the newest version (1:1.2.11.dfsg-0ubuntu2.2).
zlib1g-dev set to manually installed.
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  swig3.0
Suggested packages:
  swig-doc swig-examples swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  swig swig3.0
0 upgraded, 2 newly installed, 0 to remove and 21 not upgraded.
Need to get 1,100 kB of archives.
After this operation, 5,822 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig3.0 amd64 3.0.12-1 [1,094 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig amd64 3.0.12-1 [

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 35.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for imageio: filename=imageio-2.4.1-py3-none-any.whl size=3303886 sha256=a3a5434dcf0edc00a387119cf4453513657b67b0bf1de65d2b67ace4751cf117
  Stored in directory: /root/.cache/pip/wheels/be/7b/04/4d8d56f1d503e5c404f0de6018c0cfa592c71588a39b49e002
Successfully built imageio
  Attempting uninstall: imageio
    Found existing installation: imageio 2.23.0
    Uninstalling imageio-2.23.0:
      Successfully uninstalled imageio-2.23.0
Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.11-0ubuntu0.1).
The following package was automatically installed and is no longer required:
  

In [ ]:
from typing import Optional

import gym
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
from tqdm import tqdm

from colabgymrender.recorder import Recorder


Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)2949120/45929032 bytes (6.4%)7274496/45929032 bytes (15.8%)11427840/45929032 bytes (24.9%)15728640/45929032 bytes (34.2%)20013056/45929032 bytes (43.6%)24313856/45929032 bytes (52.9%)28557312/45929032 bytes (62.2%)32858112/45929032 bytes (71.5%)37175296/45929032 bytes (80.9%)41459712/45929032 bytes (90.3%)45801472/45929032 bytes (99.7%)45929032/45929032 bytes (100.0%)
  Done
File saved as /root

In [ ]:
class ActorCriticController:
    def __init__(self, environment, learning_rate: float, discount_factor: float, \
                 input_size: int, output_size: int, separate_nets=False, small_net=False) -> None:
        # dodatkowe parametry:
        self.small_net = small_net
        self.separate_nets = separate_nets
        self.input_size = input_size
        self.output_size = output_size
        # 
        self.environment = environment
        self.discount_factor: float = discount_factor
        self.model: tf.keras.Model = self.create_actor_critic_model()
         # TODO: przygotuj odpowiedni optymizator, pamiętaj o learning rate!
        self.optimizer: tf.keras.optimizers.Optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate) # Adam jest optymalizatorem gradientowym
        # self.model.compile(self.optimizer) # compilacja modelu z wybranym optymalizatorem 
        self.log_action_probability: Optional[tf.Tensor] = None  # zmienna pomocnicza, przyda się do obliczania docelowej straty
        self.tape: Optional[tf.GradientTape] = None  # zmienna pomocnicza, związana z działaniem frameworku
        self.last_error_squared: float = 0.0  # zmienna używana do wizualizacji wyników

    def create_actor_critic_model(self) -> tf.keras.Model:
        # TODO: przygotuj potrzebne warstwy sieci neuronowej o odpowiednich aktywacjach i rozmiarach

        if self.small_net:
            H1 = 128
            H2 = 32
        else:
            H1 = 1024
            H2 = 256 

        x_input = tf.keras.layers.Input(self.input_size)

        # galaz wspolna
        x = tf.keras.layers.Dense(H1, activation='relu')(x_input)
        x = tf.keras.layers.LayerNormalization()(x)
        x = tf.keras.layers.Dense(H2, activation='relu')(x)
        x = tf.keras.layers.LayerNormalization()(x)

        if self.separate_nets: # jesli wykorzystujemy  oddzielna siec to tworzymy oddzielna siec 
            y = tf.keras.layers.Dense(H1, activation='relu')(x_input)
            y = tf.keras.layers.LayerNormalization()(y)
            y = tf.keras.layers.Dense(H2, activation='relu')(y)
            y = tf.keras.layers.LayerNormalization()(y)

            actor_output = tf.keras.layers.Dense(self.output_size, activation="softmax")(x)
            critic_output = tf.keras.layers.Dense(1, activation="linear")(y)
        else:
            actor_output = tf.keras.layers.Dense(self.output_size, activation="softmax")(x)
            critic_output = tf.keras.layers.Dense(1, activation="linear")(x)

        # actor_output = tf.keras.layers.Dense(4, activation="softmax")(actor_x)
        # critic_output = tf.keras.layers.Dense(1, activation="linear")(critic_x)

        return tf.keras.Model(inputs=x_input, outputs=[actor_output, critic_output])

    def choose_action(self, state: np.ndarray) -> int:
        state = self.format_state(state)  # przygotowanie stanu do formatu akceptowanego przez framework

        self.tape = tf.GradientTape()
        with self.tape:
            # wszystko co dzieje się w kontekście danej taśmy jest zapisywane i może posłużyć do późniejszego wyliczania pożądanych gradientów
            # action = None  # TODO: tu trzeba wybrać odpowiednią akcję korzystając z aktora
            probs, val = self.model(state)
            actions_distribution = tfp.distributions.Categorical(probs=probs)
            action = actions_distribution.sample(1)
            # self.log_action_probability = None  # TODO: tu trzeba zapisać do późniejszego wykorzystania logarytm prawdopodobieństwa wybrania uprzednio wybranej akcji (będzie nam potrzebny by policzyć stratę aktora)
            self.log_action_probability = tf.math.log(probs[0, int(action)])
            self.curr_value = val
        return int(action)

    # noinspection PyTypeChecker
    def learn(self, state: np.ndarray, reward: float, new_state: np.ndarray, terminal: bool) -> None:
        state = self.format_state(state)
        new_state = self.format_state(new_state)

        with self.tape:  # to ta sama taśma, które użyliśmy już w fazie wybierania akcji
            # wszystko co dzieje się w kontekście danej taśmy jest zapisywane i może posłużyć do późniejszego wyliczania pożądanych gradientów
            pred = self.model(new_state)[1]

            # error = None  # TODO: tu trzeba obliczyć błąd wartościowania aktualnego krytyka
            if terminal:
                error = reward - self.curr_value
            else:
                error = reward + self.discount_factor * pred - self.curr_value

            self.last_error_squared = float(error) ** 2

            # loss = None  # TODO: tu trzeba obliczyć sumę strat dla aktora i krytyka
            loss = error ** 2 - error * self.log_action_probability

        gradients = self.tape.gradient(loss, self.model.trainable_weights)  # tu obliczamy gradienty po wagach z naszej straty, pomagają w tym informacje zapisane na taśmie
        self.optimizer.apply_gradients(zip(gradients, self.model.trainable_weights))  # tutaj zmieniamy wagi modelu wykonując krok po gradiencie w kierunku minimalizacji straty

    @staticmethod
    def format_state(state: np.ndarray) -> np.ndarray:
        return np.reshape(state, (1, state.size))



In [ ]:
def main() -> None:
    #environment = gym.make('CartPole-v1')  # zamień na gym.make('LunarLander-v2') by zająć się lądownikiem
    environment = gym.make('LunarLander-v2')
    # controller = ActorCriticController(environment, 0.0001, 0.99, \
    #                                    input_size=4, output_size=2, small_net=False, separate_nets=True)
    # controller = ActorCriticController(environment, 0.0001, 0.99, \
    #                                    input_size=4, output_size=2, small_net=False, separate_nets=False)
    # controller = ActorCriticController(environment, 0.0001, 0.99, \
    #                                    input_size=4, output_size=2, small_net=True, separate_nets=True)
    # controller = ActorCriticController(environment, 0.01, 0.99, \
    #                                    input_size=4, output_size=2, small_net=True, separate_nets=True)
    controller = ActorCriticController(environment, 0.0001, 0.99, \
                                       input_size=8, output_size=4, small_net=False, separate_nets=True)

    video_dir = "drive/MyDrive/UM_LAB2_files/scenario6/video"
    environment = Recorder(environment, video_dir) 

    # scenarios 1-4 zawierają CartPole-v1, następne zawierają już problem lądownika

    past_rewards = []
    past_errors = []
    for i_episode in tqdm(range(2000)):  # tu decydujemy o liczbie epizodów
        done = False
        state = environment.reset()
        reward_sum = 0.0
        errors_history = []

        while not done:
            # environment.render()  # tą linijkę możemy wykomentować, jeżeli nie chcemy mieć wizualizacji na żywo

            action = controller.choose_action(state)
            new_state, reward, done, info = environment.step(action)
            controller.learn(state, reward, new_state, done)
            state = new_state
            reward_sum += reward
            errors_history.append(controller.last_error_squared)


        past_rewards.append(reward_sum)
        past_errors.append(np.mean(errors_history))

        window_size = 50  # tutaj o rozmiarze okienka od średniej kroczącej
        if i_episode % 25 == 0:  # tutaj o częstotliwości zrzucania wykresów
            if len(past_rewards) >= window_size:
                fig, axs = plt.subplots(2)
                axs[0].plot(
                    [np.mean(past_errors[i:i + window_size]) for i in range(len(past_errors) - window_size)],
                    'tab:red',
                )
                axs[0].set_title('mean squared error')
                axs[1].plot(
                    [np.mean(past_rewards[i:i+window_size]) for i in range(len(past_rewards) - window_size)],
                    'tab:green',
                )
                axs[1].set_title('sum of rewards')
            plt.savefig(f'drive/MyDrive/UM_LAB2_files/scenario6/plots/learning_{i_episode}.png')
            plt.clf()
            controller.model.save_weights("drive/MyDrive/UM_LAB2_files/scenario6/weights/weights_scenario6")

    environment.close()
    # controller.model.save("final.model")  # tu zapisujemy model
    controller.model.save_weights("drive/MyDrive/UM_LAB2_files/scenario6/weights/weightscenario6_final")
    controller.model.save("drive/MyDrive/UM_LAB2_files/scenario6/models/scenario6.model")


In [ ]:
if __name__ == '__main__':
    main()


  deprecation(

  deprecation(

See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(

 26%|██▋       | 525/2000 [3:45:44<17:46:26, 43.38s/it]WARNING:py.warnings:<ipython-input-4-ba95db345af5>:45: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, axs = plt.subplots(2)

 45%|████▌     | 904/2000 [7:54:46<14:10:03, 46.54s/it]